In [2]:
import pandas as pd
import json
import tqdm

In [3]:
keyNamePath='data/key_name.csv'
keyTestTConfigPath='data/key_test_t_config.csv'
keyTestTScenesPath='data/key_test_t_scenes.csv'
# keyTestTPath='data/key_test_t.csv'
keyTestTPath='backup/forTest/keyTestT.csv'

In [4]:
keyTestT = pd.read_csv(keyTestTPath, usecols=['dimension', 'results'])

In [5]:
keyTestT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   results    50 non-null     object
 1   dimension  50 non-null     object
dtypes: object(2)
memory usage: 928.0+ bytes


In [6]:
keyTestT.head()

,results,dimension
0,"{""#wrk_qps_avg"": 1547599.39, ""wrk_latency_avg""...","{""cvm_cpu"": ""62"", ""platform"": ""qcloud"", ""test_..."
1,"{""openssl_des_size_16"": 27246690.0, ""openssl_d...","{""cvm_cpu"": ""32"", ""platform"": ""qcloud"", ""test_..."
2,"{""#ls_time"": 18}","{""dd"": ""name_11"", ""cvm_cpu"": ""1"", ""platform"": ..."
3,"{""#ls_time"": 18}","{""dd"": ""name_11"", ""cvm_cpu"": ""1"", ""platform"": ..."
4,"{""#mlc_idle_latency"": 90.6}","{""cvm_cpu"": ""8"", ""platform"": ""default"", ""test_..."


In [7]:
resultArgList=[]
dimensionArgList=[]

In [8]:
def getResultArgList(resultArgList):
    # 解析results列的json数据，将每个json数据中带有#的数据提取出来，放入inputlist中，使用tqdm显示进度条
    errornum = 0
    for index, row in tqdm.tqdm(keyTestT.iterrows(), total=keyTestT.shape[0]):
        result = row['results']
        try:
            resultJson = json.loads(result)
            for key in resultJson:
                if key.find('#') != -1:
                    resultArgList.append(key)
        except:
            errornum += 1

    print('json解析错误的数量：', errornum)
    # 去重
    resultArgList = list(set(resultArgList))
    # 指标个数
    print('指标个数：', len(resultArgList))
    # 打印前五个指标
    print('前五个指标：', resultArgList[:5])
    return resultArgList

In [9]:
resultArgList = getResultArgList(resultArgList)

100%|██████████| 50/50 [00:00<00:00, 5914.13it/s]

json解析错误的数量： 0
指标个数： 69
前五个指标： ['#mlc_idle_latency', '#openssl_sha256_size_16', '#openssl_aes_256_size_16384', '#ping_min', '#stress_ng_real_time_ops_per_s']


In [10]:
# 打印outputRes的前三个元素
def showVec(outputRes,num=3):
    for i in range(len(outputRes)):
        if i<num:
            print(outputRes[i])
        else:
            break

In [11]:
def getAllKeys(inputJson):
    for key in inputJson:
        if isinstance(inputJson[key], dict):
            getAllKeys(inputJson[key])  # 递归调用
        else:
            return key

In [12]:
def getInputFeature(keyTestT):
    errornum = 0
    # 指定需要提取的字段
    res=[]
    keys_to_extract = ['cvm_cpu', 'cvm_memory', 'cvm_cpu_qos', 'cvm_os_type']
    for index, row in tqdm.tqdm(keyTestT.iterrows(), total=keyTestT.shape[0]):
        dimension = row['dimension']
        # templist=[]
        templist=['cvm_cpu', 'cvm_memory', 'cvm_cpu_qos', 'cvm_os_type']
        try:
            dimensionJson = json.loads(dimension)
            for key in dimensionJson:
                if key in keys_to_extract:  # 判断该字段是否需要提取
                    if isinstance(dimensionJson[key], dict):
                        key_value = getAllKeys(dimensionJson[key])
                    else:
                        key_value = dimensionJson[key]
                    if key=='cvm_memory':
                        key_value=float(key_value.split(' ')[0])
                    elif key=='cvm_cpu':
                        key_value=float(key_value)
                    elif key=='cvm_cpu_qos':
                        key_value=1.0
                    templist[keys_to_extract.index(key)]=key_value
            res.append(templist)
        except:
            errornum += 1
    print('json解析错误的数量：', errornum)
    return res

In [13]:
# def getDimensionArgList(dimensionArgList):
#     # 解析dimension列的json数据，将每个字段都提取出来，放入resultArgList中，使用tqdm显示进度条
#     errornum = 0
#     for index, row in tqdm.tqdm(keyTestT.iterrows(), total=keyTestT.shape[0]):
#         dimension = row['dimension']
#         try:
#             dimensionJson = json.loads(dimension)
#             for key in dimensionJson:
#                 if isinstance(dimensionJson[key], dict):
#                     key = getAllKeys(dimensionJson[key])
#                 else:
#                     key = dimensionJson[key]
#         except:
#             errornum += 1
#     print('json解析错误的数量：', errornum)
#     # 去重
#     dimensionArgList = list(set(dimensionArgList))
#     # 指标个数
#     print('指标个数：', len(dimensionArgList))
#     # 打印前五个指标
#     print('前五个指标：', dimensionArgList[:5])
#     return dimensionArgList

In [14]:
inputDim = getInputFeature(keyTestT)

100%|██████████| 50/50 [00:00<00:00, 4163.41it/s]

json解析错误的数量： 0


In [15]:
showVec(inputDim,5)

[62.0, 214.0, 1.0, 'CentOS Linux release 8.2.2004 (Core)']
[32.0, 64.0, 1.0, 'CentOS Linux release 8.2.2004 (Core)']
[1.0, 2.0, 'cvm_cpu_qos', 'cvm_os_type']
[1.0, 2.0, 'cvm_cpu_qos', 'cvm_os_type']
[8.0, 16.0, 'cvm_cpu_qos', 'CentOS Linux release 7.2 (Final)']


In [16]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np

In [32]:
def getInput(inputDim):
    labelencoder = LabelEncoder()
    onehotencoder = OneHotEncoder()
    data = np.array(inputDim)[:, :4]
    encoders = []
    for i in range(data.shape[1]):
        encoder = LabelEncoder()
        encoder.fit(data[:, i])
        data[:, i] = encoder.transform(data[:, i])
        encoders.append(encoder)

    data = data.astype(float)
    return data

In [33]:
input=getInput(inputDim)

In [36]:
print(input.shape)
showVec(input,5)

(50, 4)
[7. 4. 0. 4.]
[ 4. 10.  0.  4.]
[0. 2. 1. 8.]
[0. 2. 1. 8.]
[9. 0. 1. 0.]


In [19]:
def showDict(dict,num=3):
    # 打印字典的前三个元素
    for i, (key, value) in enumerate(dict.items()):
        if i<num:
            print(f"{key}: {value}")
        else:
            break

In [20]:
# 将resultArgList列表转换为字典，key为指标名称，value为空字符串
resDict = dict(zip(resultArgList, [0] * len(resultArgList)))

In [21]:
showDict(resDict)

#mlc_idle_latency: 0
#openssl_sha256_size_16: 0
#openssl_aes_256_size_16384: 0


In [22]:
# 将dimensionArgList列表转换为字典，key为指标名称，value为空字符串
dimDict = dict(zip(dimensionArgList, [0] * len(dimensionArgList)))

In [23]:
showDict(dimDict)

In [24]:
# 获得keyTestT中的results列的json数据
def getOutput(keyTestT, column, inputStandardDict):
    errornum = 0
    res=[]
    for index, row in tqdm.tqdm(keyTestT.iterrows(), total=keyTestT.shape[0]):
        data = row[column]
        resDict = inputStandardDict.copy()
        try:
            jsonData = json.loads(data)
            for key in resDict.keys():
                if key in jsonData:
                    # 判断jsonData[key]是否为合法数字，若是合法数字，则将其赋值给resDict[key]
                    if isinstance(jsonData[key], (int, float)):
                        resDict[key] = float(jsonData[key])
        except:
            errornum += 1
        # 将resDict中所有的value值提取出来，放入resFeature中
        resFeature = list(resDict.values())
        res.append(resFeature)
    print('json解析错误的数量：', errornum)
    return res

In [25]:
output=getOutput(keyTestT, 'results', resDict)

100%|██████████| 50/50 [00:00<00:00, 4165.48it/s]

json解析错误的数量： 0


In [26]:
showVec(output,10)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1547599.39, 0, 0, 0, 0, 0]
[0, 186794640.0, 169255520.0, 0, 0, 169277510.0, 53858.2, 1319133930.0, 98599460.0, 688029040.0, 169156.8, 169084740.0, 0, 0, 220491340.0, 0, 0, 0, 1138802430.0, 0, 0, 231.1, 10220.5, 0, 0, 1245835510.0, 523630360.0, 684749490.0, 0, 0, 168236760.0, 0, 1254420620.0, 5522915560.0, 216549540.0, 0, 166032790.0, 0, 0, 0, 0, 304494350.0, 876027720.0, 1564.2, 138796040.0, 0, 0, 221953720.0, 0, 0, 0, 221617310.0, 4550588950.0, 463084970.0, 156977930.0, 0, 0, 0, 0, 379233260.0, 221211400.0, 0, 0, 0, 0, 201371230.0, 14860.8, 25034650.0, 640550830.0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 18.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0